In [1]:
from wrappers import Arctic, GPT

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

from pydantic import BaseModel, Field

In [2]:
class Agent(BaseModel):
    name: str
    description: str = Field(..., title="Description of the agent")

    def __str__(self):
        return self.name, self.description
    
consultant = Agent(name="Consultant", description="An agent to use when you need additional information or advice on a topic.")
sql_generator = Agent(name="SQL Generator", description="An agent to use when you need to generate any SQL code.")
db_agent = Agent(name="Database Agent", description="An agent to use when you need to interact with the app database, which user's may want to modify.")

agents = [consultant, sql_generator, db_agent]

In [3]:
options = ["SELF"] + [agent.name for agent in agents]

# Assumptions:
# - Only ONE agent can be assigned to a request
# - Only ONE request is being handled at a time 
# (if we have multiple requests, we would need to loop through them with a GPT orchestrator calling this chain)
chain = (
    PromptTemplate.from_template(
        f"""
        You are a supervisor, tasked with managing user requests while having access to the
        following members on your team: {agents}.

        Given the following user request, classify it as being a request you could either fulfill yourself, or that
        one of your team members could better handle. 

        Select one of: {options}"""
        + """
        User Request: {request}
        """
    )
    | GPT(model="gpt-3.5-turbo")
    | StrOutputParser()
)

# example to test validity of the chain
chain.invoke({"request": "Add 2 + 2."})

'SELF'

Now, we create sub-chains to route to based on the output - SELF, Consultant,  or SQL Generator.

In [4]:
# self-chain
self_chain = (
    PromptTemplate.from_template(
        "{request}"
    )
    | GPT(model="gpt-3.5-turbo")
    | StrOutputParser()
)

# consultant chain
consultant_chain = (
    PromptTemplate.from_template(
        """You are a helpful assistant. You have access to tools and resources that can help you provide
        additional information or advice on a topic. Given the following user request, provide a response that
        would be helpful to the user.

        Request: {request}"""
    )
    | GPT(model="gpt-3.5-turbo")
    | StrOutputParser()
)

# SQL Generator chain
sql_generator_chain = (
    PromptTemplate.from_template(
        """You are an expert data engineer that is very proficient in SQL. Given the following user request,
        generate the SQL code that would fulfill the user's request.

        Request: {request}"""
    )
    | Arctic()
    | StrOutputParser()
)

db_chain = (
    RunnableLambda(lambda x: f"Added {x} to the database.")
)

In [5]:
def route(msg: str):

    print(f"""Routed message {msg["request"]} to {msg["topic"]} agent.""")

    if "self" in msg["topic"].lower():
        return self_chain
    elif "consultant" in msg["topic"].lower():
        return consultant_chain
    elif "sql generator" in msg["topic"].lower():
        return sql_generator_chain
    elif "database agent" in msg["topic"].lower():
        return db_chain
    else:
        raise ValueError(f"Invalid agent {msg} selected.")

full_chain = {"request": lambda x: x["request"], "topic": lambda x: chain.invoke(x)} | RunnableLambda(route)

In [6]:
full_chain.invoke({"request": "What is the capital of France?"})

Routed message What is the capital of France? to SELF agent.


'The capital of France is Paris.'

In [7]:
full_chain.invoke({"request": "Implement fizzbuzz in SQL."})

Routed message Implement fizzbuzz in SQL. to SQL Generator agent.


"Sure, here's a simple implementation of FizzBuzz in SQL:\n\n```sql\nWITH RECURSIVE fizzbuzz AS (\n    SELECT 1 as num\n    UNION ALL\n    SELECT num + 1 FROM fizzbuzz WHERE num < 100\n)\nSELECT \n    CASE \n        WHEN MOD(num, 3) = 0 AND MOD(num, 5) = 0 THEN 'FizzBuzz'\n        WHEN MOD(num, 3) = 0 THEN 'Fizz'\n        WHEN MOD(num, 5) = 0 THEN 'Buzz'\n        ELSE CAST(num AS VARCHAR)\n    END\nFROM fizzbuzz;\n```\nThis script creates a recursive common table expression (CTE) that generates numbers from 1 to 100. Then it uses a `CASE` statement to print 'Fizz', 'Buzz', 'FizzBuzz', or the number itself based on the rules of FizzBuzz game."

In [8]:
full_chain.invoke({"request": "Add this customer's information to the database."})

Routed message Add this customer's information to the database. to Database Agent agent.


'Added {\'request\': "Add this customer\'s information to the database.", \'topic\': \'Database Agent\'} to the database.'

In [9]:
# wrap ReAct agent around arctic and gpt and see if it works
# if so, substitute the llms with the react agent in every chain